In [11]:
pip install numpy==1.23.4

Note: you may need to restart the kernel to use updated packages.


In [2600]:
import numpy as np
import pandas as pd
import cobra.test
import os

from os.path import join

from time import time


from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

import cobra
from cobra.test import create_test_model
cobra_config = cobra.Configuration()
cobra_config.solver = "gurobi"
model = create_test_model("textbook")
model.solver

a = cobra.io.read_sbml_model("iEC1344_C.xml")


In [2601]:
int_reactions= list()
i= -1
r = a.reactions[0]
r_id = r.id
zero= list()
one= list()
one_at_zero= list()

for r in a.reactions:  #finds reactions that have one feasible value at flux=0 or for another flux
    i= i+1
    r = a.reactions[i]
    r_id = r.id

    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    v= len(csvFile["Biomass"])-csvFile["Biomass"].isna().sum()

    flt= float(csvFile.iloc[300,  2])
    
    if v == 1 and csvFile.iloc[300,  2] != flt:
        one.append(r_id)
    elif v == 0:
        zero.append(r_id)
        
    if v == 1 and csvFile.iloc[300,  2] == flt:
        one_at_zero.append(r_id)

print(one)
print(zero)
print(one_at_zero)

D= pd.DataFrame (one, columns = ['1 feasible value'])
D_at_zero= pd.DataFrame (one_at_zero, columns = ['1 feasible value_at_zero'])


[]
[]
['EX_cm_e', 'EX_cobalt2_e', 'EX_colipa_e', 'EX_colipap_e', 'EX_crn_e', 'EX_crn__D_e', 'DM_4crsol_c', 'EX_cu2_e', 'EX_cyan_e', 'DM_aacald_c', 'DM_amob_c', 'DM_mththf_c', 'EX_glyb_e', 'EX_gthox_e', 'BIOMASS_Ec_iJO1366_WT_53p95M', 'EX_14glucan_e', 'EX_h2o2_e', 'EX_hacolipa_e', 'EX_halipa_e', 'EX_34dhpac_e', 'EX_hg2_e', 'EX_4hoxpacd_e', 'EX_cys__D_e', 'EX_acgal_e', 'EX_inost_e', 'EX_acolipa_e', 'EX_adocbl_e', 'EX_ag_e', 'EX_lipoate_e', 'EX_ala_B_e', 'EX_malthx_e', 'EX_maltpt_e', 'EX_malttr_e', 'EX_maltttr_e', 'EX_arbt_e', 'EX_meoh_e', 'EX_met__D_e', 'EX_aso3_e', 'EX_mg2_e', 'EX_mincyc_e', 'EX_btn_e', 'EX_mmet_e', 'EX_mn2_e', 'EX_ca2_e', 'EX_cbi_e', 'EX_mobd_e', 'EX_n2o_e', 'EX_na1_e', 'EX_nac_e', 'EX_cbl1_e', 'EX_cd2_e', 'EX_chol_e', 'EX_cl_e', 'EX_ni2_e', 'EX_nmn_e', 'EX_dms_e', 'EX_novbcn_e', 'EX_o16a4colipa_e', 'EX_doxrbcn_e', 'EX_o2s_e', 'EX_xtsn_e', 'EX_pacald_e', 'EX_zn2_e', '12DGR120tipp', '12DGR140tipp', '12DGR141tipp', '12DGR160tipp', 'EX_pnto__R_e', '12DGR161tipp', '12DGR18

In [2602]:
i= 0
r = a.reactions[0]
l2= list()
l3= list()

for r in a.reactions:            #separates reactions that have biomass values for positive and negative flux values
    
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

    i3= 301

    while i3>300 and i3<len(csvFile):
        inf= csvFile.iloc[i3,  2]
        flt= float(csvFile.iloc[i3,  2])
    
        if csvFile.iloc[i3,  2] == flt and r_id not in l2:
            l2.append(r_id)     #l2 contains reactions that have biomass values for positive flux values
        i3= i3+1
    
    i4= 0
    
    while i4<300:
        inf= csvFile.iloc[i4,  2]
        flt= float(csvFile.iloc[i4,  2])
    
        if csvFile.iloc[i4,  2] == flt and r_id not in l3:
            l3.append(r_id)    #l3 contains reactions that have biomass values for negative flux values
        i4= i4+1

    i= i+1

D1= pd.DataFrame (l2, columns = ['Reaction names'])
D2= pd.DataFrame (l3, columns = ['Reaction names'])
b= D1.iloc[:, 0]
c= D2.iloc[:, 0]

union = pd.Series(np.union1d(b, c))
intersect = pd.Series(np.intersect1d(b, c))
bigb2= union[~union.isin(intersect)]

D3= pd.DataFrame (bigb2, columns = ['Reaction names'])
D3
D3.to_csv("res2 lactose.csv")      #contains reactions that have biomass values for either positive or negative flux values


In [2603]:
i= 0
indecr_and_stable_above_zero= list()
diff_indecr_above_zero= list()
flat_above_zero= list()
flat2_above_zero= list()
flat3_above_zero= list()
r_id = a.reactions[0]


for r_id in a.reactions:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat
    l= list()
    n=0
    n2=0
    count=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

    while n < 600:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])
    d.Biomass = d.Biomass.round(2)
    
    
    my_series = d['Biomass'].squeeze()


    if len(d)> 30 and r_id in l2 and r_id not in l3:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 0.05*len(d) and count< 0.95*len(d):
            indecr_and_stable_above_zero.append(r_id)

        elif count> 0.95*len(d):
            flat_above_zero.append(r_id)
        
        else:
            diff_indecr_above_zero.append(r_id)

    
    if len(d)> 1 and len(d)<30  and r_id in l2 and r_id not in l3:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 3 and count< len(d)-4:
            indecr_and_stable_above_zero.append(r_id)

        elif count> 1 and len(d) < 3:
            flat_above_zero.append(r_id)
        
        elif count> 2 and len(d) < 4:
            flat2_above_zero.append(r_id)
        
        elif count> len(d)-4 and len(d)>4:
            flat3_above_zero.append(r_id)
        
        else:
            diff_indecr_above_zero.append(r_id)
    
    i= i+1

flat_above_zero
diff_indecr_above_zero
indecr_and_stable_above_zero


['2MAHMP',
 'AACPS8',
 'AACPS9',
 'ABTA',
 'ACCOAC',
 'ACONTa',
 'ACONTb',
 'ACPPAT120',
 'ACPPAT140',
 'ACPPAT141',
 'ACPPAT160',
 'ACPPAT161',
 'ACPPAT180',
 'ACPPAT181',
 'ALAALAr',
 'ALAR',
 'ADNCYC',
 'AP4AH',
 'AP4AS',
 'CMtex',
 'CD2tpp',
 'COBALT2tpp',
 'CS',
 'DASYN120',
 'DASYN140',
 'DASYN141',
 'DASYN160',
 'CU2abcpp',
 'DASYN161',
 'DASYN180',
 'DASYN181',
 'CU2tpp',
 'DOXRBCNtex',
 'ARGORNt7pp',
 'DTMPK',
 'EDA',
 'EDD',
 'DURADx',
 'ARGt3pp',
 'FACOAE60',
 'FBA3',
 'FHL',
 'FORtppi',
 'FUSAtex',
 'GK1',
 'G5SD',
 'G6PDH2r',
 'GLCDpp',
 'GLCNt2rpp',
 'GALUi',
 'GLNS',
 'GLU5K',
 'GF6PTA',
 'HYPOE',
 'HCO3E',
 'ICDHyr',
 'HEX4',
 'LCTStpp',
 'LYSt3pp',
 'MCOATA',
 'MALt3pp',
 'MAN1PT2',
 'MINCYCtex',
 'MELIBt2pp',
 'MELIBt3ipp',
 'MN2t3pp',
 'NADDP',
 'NADK',
 'NADPPPS',
 'GSPMDA',
 'GSPMDS',
 'NDPK2',
 'NDPK4',
 'NOVBCNtex',
 'NH4tpp',
 'NTD4',
 'NTD5',
 'GTPDPK',
 'O2tex',
 'P5CR',
 'GUACYC',
 'PGCD',
 'PDE1',
 'PDE4',
 'PDXPP',
 'PSERT',
 'PSP_L',
 'RBK',
 'RFAMPtex',
 

In [2604]:
i= 0
indecr_and_stable_below_zero= list()
diff_indecr_below_zero= list()
flat_below_zero= list()
flat2_below_zero= list()
flat3_below_zero= list()
r_id = a.reactions[0]


for r_id in a.reactions:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat
    l= list()
    n=0
    n2=0
    count=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

    while n < 600:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])
    d.Biomass = d.Biomass.round(2)
    
    my_series = d['Biomass'].squeeze()
    

    if len(d)> 30 and r_id in l3 and r_id not in l2:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 0.05*len(d) and count< 0.95*len(d):
            indecr_and_stable_below_zero.append(r_id)

        elif count> 0.95*len(d):
            flat_below_zero.append(r_id)
        
        else:
            diff_indecr_below_zero.append(r_id)

    
    if len(d)> 1 and len(d)<30  and r_id in l3 and r_id not in l2:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 3 and count< len(d)-4:
            indecr_and_stable_below_zero.append(r_id)

        elif count> 1 and len(d) < 3:
            flat_below_zero.append(r_id)
        
        elif count> 2 and len(d) < 4:
            flat2_below_zero.append(r_id)
        
        elif count> len(d)-4 and len(d)>4:
            flat3_below_zero.append(r_id)
        
        else:
            diff_indecr_below_zero.append(r_id)
    
    i= i+1

flat_below_zero
diff_indecr_below_zero
indecr_and_stable_below_zero


['EX_csn_e',
 'EX_cynt_e',
 'EX_no_e',
 'EX_dmso_e',
 'EX_no3_e',
 'EX_o2_e',
 'EX_tmao_e',
 '2DGULRGx',
 '2DGULRGy',
 '2DGULRx',
 'ADK4',
 'ASPTA',
 'ATHRDHr',
 'CHLt2pp',
 'BETALDHx',
 'BETALDHy',
 'BSORx',
 'BSORy',
 'CTBTt2rpp',
 'CUt3',
 'DKGLCNR2y',
 'DOGULNR',
 'ASNt2rpp',
 'FRUpts2pp',
 'FUMt2_2pp',
 'FUMt2_3pp',
 'G2PPpp',
 'G3PD2',
 'ASPO6',
 'GLNabcpp',
 'GALt2pp',
 'GLUNpp',
 'GLYBt2pp',
 'GUAtpp',
 'HEXt2rpp',
 'HG2t3pp',
 'ILEt2rpp',
 'HISt2rpp',
 'LACZ',
 'LEUt2rpp',
 'LIPOCT',
 'MLTP1',
 'MLTP2',
 'MLTP3',
 'MG2t3_2pp',
 'NODOx',
 'NODOy',
 'NO3R1bpp',
 'NO3R2bpp',
 'OMMBLHX',
 'OMMBLHX3',
 'OMPHHX',
 'OMPHHX3',
 'OPHHX',
 'OPHHX3',
 'PGMT',
 'PTRCORNt7pp',
 'SUCFUMtpp',
 'SUCTARTtpp',
 'TARTD',
 'TREptspp',
 'PMANM',
 'XANtpp',
 'VALt2rpp',
 'MECDPDH2',
 'XYLt2pp',
 '2DGLCNRx',
 '2DGLCNRy',
 'DOGULND',
 'TARTDC']

In [2605]:
i2= 0
r_id = indecr_and_stable_above_zero[0]
mon_incr_and_stable_above_zero= list()
mon_decr_and_stable_above_zero= list()

for r_id in indecr_and_stable_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = indecr_and_stable_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            mon_decr_and_stable_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            mon_incr_and_stable_above_zero.append(r_id)

    
    i2= i2 + 1


set1= set(indecr_and_stable_above_zero)
set2= set(mon_decr_and_stable_above_zero)
set3= set(mon_incr_and_stable_above_zero)
inANDdecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_above_zero


['ACCOAC',
 'COBALT2tpp',
 'DASYN160',
 'ACPPAT160',
 'PMPK',
 'ACONTb',
 'DASYN161',
 'PSERT',
 'ACPPAT161',
 'NADK',
 'TMPK',
 'CU2tpp',
 'GF6PTA',
 'GK1',
 'PSP_L',
 'MCOATA',
 'ACONTa',
 'ALAR',
 'HCO3E',
 'PGCD',
 'ALAALAr',
 'CS',
 'NH4tpp',
 'DTMPK',
 'NDPK4',
 'P5CR',
 'ICDHyr',
 'UMPK',
 'GLNS',
 'NDPK2']

In [2606]:
w= list()
w2= list()
i2=0

while i2 < len(indecr_and_stable_above_zero):
    
    i=0
    r_id = indecr_and_stable_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w2.append(r_id)

    
    i2= i2+1

w


['ACCOAC',
 'ACPPAT160',
 'ACPPAT161',
 'ALAALAr',
 'ALAR',
 'COBALT2tpp',
 'DASYN160',
 'DASYN161',
 'CU2tpp',
 'DTMPK',
 'GK1',
 'GF6PTA',
 'MCOATA',
 'NADK',
 'NDPK4',
 'P5CR',
 'TMPK',
 'PMPK']

In [2607]:
inANDdecr_and_stable_above_zero.remove("ACCOAC")
mon_decr_and_stable_above_zero.append("ACCOAC")

inANDdecr_and_stable_above_zero.remove("ACPPAT160")
mon_decr_and_stable_above_zero.append("ACPPAT160")

inANDdecr_and_stable_above_zero.remove("ACPPAT161")
mon_decr_and_stable_above_zero.append("ACPPAT161")

inANDdecr_and_stable_above_zero.remove("ALAALAr")
mon_decr_and_stable_above_zero.append("ALAALAr")

inANDdecr_and_stable_above_zero.remove("ALAR")
mon_decr_and_stable_above_zero.append("ALAR")

inANDdecr_and_stable_above_zero.remove("COBALT2tpp")
mon_decr_and_stable_above_zero.append("COBALT2tpp")

inANDdecr_and_stable_above_zero.remove("DASYN160")
mon_decr_and_stable_above_zero.append("DASYN160")

inANDdecr_and_stable_above_zero.remove("DASYN161")
mon_decr_and_stable_above_zero.append("DASYN161")

inANDdecr_and_stable_above_zero.remove("CU2tpp")
mon_decr_and_stable_above_zero.append("CU2tpp")

inANDdecr_and_stable_above_zero.remove("DTMPK")
mon_decr_and_stable_above_zero.append("DTMPK")

inANDdecr_and_stable_above_zero.remove("GK1")
mon_decr_and_stable_above_zero.append("GK1")

inANDdecr_and_stable_above_zero.remove("GF6PTA")
mon_decr_and_stable_above_zero.append("GF6PTA")

inANDdecr_and_stable_above_zero.remove("MCOATA")
mon_decr_and_stable_above_zero.append("MCOATA")

inANDdecr_and_stable_above_zero.remove("NADK")
mon_decr_and_stable_above_zero.append("NADK")

inANDdecr_and_stable_above_zero.remove("NDPK4")
mon_decr_and_stable_above_zero.append("NDPK4")

inANDdecr_and_stable_above_zero.remove("TMPK")
mon_decr_and_stable_above_zero.append("TMPK")

inANDdecr_and_stable_above_zero.remove("PMPK")
mon_decr_and_stable_above_zero.append("PMPK")

inANDdecr_and_stable_above_zero.remove("P5CR")
mon_decr_and_stable_above_zero.append("P5CR")


In [2608]:
i2= 0
r_id = indecr_and_stable_below_zero[0]
mon_incr_and_stable_below_zero= list()
mon_decr_and_stable_below_zero= list()

for r_id in indecr_and_stable_below_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = indecr_and_stable_below_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            mon_decr_and_stable_below_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            mon_incr_and_stable_below_zero.append(r_id)

    
    i2= i2 + 1


set1= set(indecr_and_stable_below_zero)
set2= set(mon_decr_and_stable_below_zero)
set3= set(mon_incr_and_stable_below_zero)
inANDdecr_and_stable_below_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero


['G3PD2', 'ASPTA']

In [2609]:
w3= list()
w4= list()
i2=0

while i2 < len(indecr_and_stable_below_zero):
    
    i=0
    r_id = indecr_and_stable_below_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w3.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w4.append(r_id)

    
    i2= i2+1

w3


[]

In [2610]:
w4

['G3PD2']

In [2611]:
inANDdecr_and_stable_below_zero.remove("G3PD2")
mon_incr_and_stable_below_zero.append("G3PD2")


In [2612]:
i= 0
flat_below_zero_and_indecr_and_stable_above_zero= list()
diff_indecr_both_positive_and_negative= list()
indecr_and_stable_below_zero_and_indecr_and_stable_above_zero= list()
indecr_and_stable_below_zero_and_flat_above_zero= list()
flat_below_zero_and_flat_above_zero= list()
diff_indecr_below_zero_and_flat_above_zero= list()
flat_below_zero_and_diff_indecr_above_zero= list()
indecr_and_stable_below_zero_and_flat_above_zero= list()
indecr_and_stable_below_zero_and_diff_indecr_above_zero= list()
flat_below_zero_and_indecr_and_stable_above_zero= list()
diff_indecr_below_zero_and_indecr_and_stable_above_zero= list()
rest_both_positive_and_negative= list()
r_id = a.reactions[0]


for r_id in a.reactions:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat
    l= list()
    n=0
    n2=0
    count=0
    count2=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

    while n < 301:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n= 300
    l4= list()
    
    while n < 600:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l4.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()


    if (len(csvFile2) + len(csvFile3))> 30 and r_id in l3 and r_id in l2:
        
        while n2 < len(csvFile2):

            if csvFile2.iloc[n2,  0] > max(csvFile2["Biomass"])-0.01 and csvFile2.iloc[n2,  0] < max(csvFile2["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        n2=0
        while n2 < len(csvFile3):

            if csvFile3.iloc[n2,  0] > max(csvFile3["Biomass"])-0.01 and csvFile3.iloc[n2,  0] < max(csvFile3["Biomass"])+0.01:
                count2= count2+1
    
            n2= n2+1
        
        if (count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and (count2> 0.05*len(csvFile3) or count2 == 0.05*len(csvFile3)) and count2< 0.95*len(csvFile3):
            indecr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)

        elif count> 0.95*len(csvFile2) and (count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and (count2> 0.95*len(csvFile3)):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)

        elif count> 0.95*len(csvFile2) and count2> 0.95*len(csvFile3):
            flat_below_zero_and_flat_above_zero.append(r_id)
        
        elif count< 0.05*len(csvFile2) and count2> 0.95*len(csvFile3):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
        
        elif count> 0.95*len(csvFile2) and count2< 0.05*len(csvFile3):
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
        
        elif (count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and count2> 0.95*len(csvFile3):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
        
        elif (count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and count2< 0.05*len(csvFile3):
            indecr_and_stable_below_zero_and_diff_indecr_above_zero.append(r_id)
        
        elif count< 0.05*len(csvFile2) and (count2== 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            diff_indecr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
            
        elif count< 0.05*len(csvFile2) and count2< 0.05*len(csvFile3):
            diff_indecr_both_positive_and_negative.append(r_id)

    
    if (len(csvFile2) + len(csvFile3))> 1 and (len(csvFile2) + len(csvFile3))<30  and r_id in l3 and r_id in l2:
        
        n2=0
        while n2 < len(csvFile2):

            if csvFile2.iloc[n2,  0] > max(csvFile2["Biomass"])-0.01 and csvFile2.iloc[n2,  0] < max(csvFile2["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        n2=0
        while n2 < len(csvFile3):

            if csvFile3.iloc[n2,  0] > max(csvFile3["Biomass"])-0.01 and csvFile3.iloc[n2,  0] < max(csvFile3["Biomass"])+0.01:
                count2= count2+1
    
            n2= n2+1
        
        if (count> 3 and count< len(csvFile2)-4) and (count2> 1 and len(csvFile3) < 3):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
        
        elif (count> 3 and count< len(csvFile2)-4) and (count2> 3 and count2< len(csvFile3)-4):
            indecr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count> 3 and count< len(csvFile2)-4) and (count2> 2 and len(csvFile3) < 4):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
        
        elif (count> 3 and count< len(csvFile2)-4) and (count2> len(csvFile3)-4 and len(csvFile3)>4):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
        
        elif (count> 1 and len(csvFile2) < 3) and (count2> 3 and count2< len(csvFile3)-4):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count> 2 and len(csvFile2) < 4) and (count2> 3 and count2< len(csvFile3)-4):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count> len(d)-4 and len(d)>4) and (count2> 3 and count2< len(csvFile3)-4):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        else:
            rest_both_positive_and_negative.append(r_id)
    
    i= i+1

flat_below_zero_and_indecr_and_stable_above_zero
rest_both_positive_and_negative
indecr_and_stable_below_zero_and_flat_above_zero
indecr_and_stable_below_zero_and_indecr_and_stable_above_zero


['ACALD',
 'ALDD2y',
 'ASPt2_2pp',
 'ASPt2_3pp',
 'CITL',
 'CMtpp',
 'CD2t3pp',
 'COBALT2t3pp',
 'CYStpp',
 'CYTBD2pp',
 'CYTBDpp',
 'CYTBO3_4pp',
 'ARBt2rpp',
 'DAAD',
 'ARBt3ipp',
 'DOXRBCNtpp',
 'F6PP',
 'FADRx',
 'FADRx2',
 'FBA',
 'FBP',
 'FE2t2pp',
 'FE2t3pp',
 'ASNS2',
 'FLDR2',
 'FE2tpp',
 'FORt2pp',
 'FUSAtpp',
 'G6PP',
 'GLCptspp',
 'GLCt2pp',
 'ASPT',
 'GLYCDx',
 'GDPDPK',
 'GDPMNH',
 'GDPTPDP',
 'GUAt2pp',
 'GLYCLTDx',
 'GLYCLTDy',
 'IDOND2',
 'GLYK',
 'HPYRRy',
 'Kt2pp',
 'LCTSt3ipp',
 'LYSt2pp',
 'MACPD',
 'MALS',
 'MALt2_2pp',
 'MALt2_3pp',
 'MINCYCtpp',
 'ME1',
 'ME2',
 'MLTG3',
 'MLTG4',
 'MLTG5',
 'MN2tpp',
 'MNt2pp',
 'MG2tpp',
 'NADTRHD',
 'NAt3_1p5pp',
 'NAt3_2pp',
 'NOVBCNtpp',
 'NI2t3pp',
 'NI2tpp',
 'NMNAT',
 'OAADC',
 'PFL',
 'PTRCt2pp',
 'RFAMPtpp',
 'PUNP1',
 'PUNP5',
 'PYK',
 'PYNP2r',
 'RPI',
 'SSALx',
 'SSALy',
 'SUCASPtpp',
 'SUCCt2_2pp',
 'SUCCt2_3pp',
 'SUCMALtpp',
 'TALA',
 'THD2pp',
 'THRA2',
 'THRt4pp',
 'TRE6PH',
 'TTRCYCtpp',
 'XANt2pp',
 'XYLI2',


In [2613]:
rest_both_positive_and_negative

[]

In [2614]:
i2= 0
r_id = indecr_and_stable_below_zero_and_indecr_and_stable_above_zero[0]
mon_incr_and_stable_both_positive_and_negative= list()
mon_decr_and_stable_both_positive_and_negative= list()

for r_id in indecr_and_stable_below_zero_and_indecr_and_stable_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = indecr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)
    
    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            mon_decr_and_stable_both_positive_and_negative.append(r_id)
        elif my_series.is_monotonic ==True:
            mon_incr_and_stable_both_positive_and_negative.append(r_id)

    
    i2= i2 + 1


set1= set(indecr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_decr_and_stable_both_positive_and_negative)
set3= set(mon_incr_and_stable_both_positive_and_negative)
inANDdecr_and_stable_both_positive_and_negative= list(set1-set2-set3)
inANDdecr_and_stable_both_positive_and_negative


['ZN2t3pp',
 'FORt2pp',
 'NMNAT',
 'PFL',
 'FADRx',
 'TRE6PH',
 'PPS',
 'THD2pp',
 'FBP',
 'SUCMALtpp',
 'FBA',
 'Kt2pp',
 'ZN2tpp',
 'GLCt2pp',
 'CYStpp',
 'FE2t3pp',
 'CITL',
 'ASNS2',
 'COBALT2t3pp',
 'MALS',
 'ACALD',
 'NI2t3pp',
 'PUNP5',
 'GDPMNH',
 'PYNP2r',
 'NI2tpp',
 'THRt4pp',
 'RPI',
 'GDPDPK',
 'ME2',
 'PPCK',
 'MALt2_2pp',
 'SSALy',
 'GLYK',
 'PYK',
 'ARBt2rpp',
 'PTRCt2pp',
 'CD2t3pp',
 'MACPD',
 'FE2tpp',
 'SUCASPtpp',
 'SUCCt2_2pp',
 'GLYCDx',
 'GLYCLTDx',
 'PPK2',
 'XYLI2',
 'GDPTPDP',
 'LYSt2pp',
 'FLDR2',
 'TALA',
 'PUNP1',
 'MN2tpp']

In [2615]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(inANDdecr_and_stable_both_positive_and_negative):
    
    i=0
    r_id = inANDdecr_and_stable_both_positive_and_negative[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 301:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 300
    l= list()
    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1
    
w5


[]

In [2616]:
i2= 0
r_id = diff_indecr_below_zero_and_flat_above_zero[0]
mon_incr_below_zero_and_flat_above_zero= list()
mon_decr_below_zero_and_flat_above_zero= list()

for r_id in diff_indecr_below_zero_and_flat_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = diff_indecr_below_zero_and_flat_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            mon_decr_below_zero_and_flat_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            mon_incr_below_zero_and_flat_above_zero.append(r_id)

    
    i2= i2 + 1


set1= set(diff_indecr_below_zero_and_flat_above_zero)
set2= set(mon_decr_below_zero_and_flat_above_zero)
set3= set(mon_incr_below_zero_and_flat_above_zero)
inANDdecr_below_zero_and_flat_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_flat_above_zero


[]

In [2617]:
mon_incr_below_zero_and_flat_above_zero

['EX_fe3_e',
 'ACACT1r',
 'ACACT2r',
 'ACACT3r',
 'ACACT4r',
 'ACACT5r',
 'ACACT6r',
 'ACACT7r',
 'FE2tex',
 'HACD1',
 'HACD2',
 'HACD3',
 'HACD4',
 'HACD5',
 'HACD6',
 'HACD7',
 'HACD8',
 'INDOLEt2rpp']

In [2618]:
i2= 0
r_id = flat_below_zero_and_diff_indecr_above_zero[0]
flat_below_zero_and_mon_incr_above_zero= list()
flat_below_zero_and_mon_decr_above_zero= list()

for r_id in flat_below_zero_and_diff_indecr_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = flat_below_zero_and_diff_indecr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            flat_below_zero_and_mon_decr_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            flat_below_zero_and_mon_incr_above_zero.append(r_id)

    
    i2= i2 + 1


set1= set(flat_below_zero_and_diff_indecr_above_zero)
set2= set(flat_below_zero_and_mon_decr_above_zero)
set3= set(flat_below_zero_and_mon_incr_above_zero)
flat_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
flat_below_zero_and_inANDdecr_above_zero


[]

In [2619]:
flat_below_zero_and_mon_decr_above_zero

['EX_lys__L_e',
 'EX_fe2_e',
 'ACACT8r_copy1',
 'ACOAD7f',
 'ADA',
 'ADD',
 'DADA',
 'CTECOAI6_copy1',
 'CTECOAI7_copy1',
 'CTECOAI8_copy1',
 'DHORD2',
 'DHORD5',
 'DHORDfum',
 'ASNS1',
 'FE3tex',
 'GLUt2rpp',
 'GLYCK',
 'GLYCK2',
 'ICHORS_copy1',
 'GLYOX3',
 'KAS14',
 'KAS15',
 'L_LACD2_copy1',
 'L_LACD3_copy1',
 'MDDCP3pp',
 'MLDCP2App',
 'MMM',
 'MICITDr_copy1',
 'NTD8',
 'NTPTP1',
 'GTPDPDP',
 'SUCOAS',
 'SERD_D',
 'SERD_L',
 'THRA',
 'THRD',
 'TRPS1',
 'TRPS2',
 'TRSARr',
 'XTSNH_copy1',
 'L_LACD3_copy2',
 'MMM2',
 'L_LACD2_copy2',
 'XTSNH_copy2']

In [2620]:
i2= 0
r_id = indecr_and_stable_below_zero_and_diff_indecr_above_zero[0]
indecr_and_stable_below_zero_and_mon_incr_above_zero= list()
indecr_and_stable_below_zero_and_mon_decr_above_zero= list()

for r_id in indecr_and_stable_below_zero_and_diff_indecr_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    n2= 300
    r_id = indecr_and_stable_below_zero_and_diff_indecr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()
    
    while n2 < 600:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            indecr_and_stable_below_zero_and_mon_decr_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            indecr_and_stable_below_zero_and_mon_incr_above_zero.append(r_id)

    
    i2= i2 + 1


set1= set(indecr_and_stable_below_zero_and_diff_indecr_above_zero)
set2= set(indecr_and_stable_below_zero_and_mon_decr_above_zero)
set3= set(indecr_and_stable_below_zero_and_mon_incr_above_zero)
indecr_and_stable_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
indecr_and_stable_below_zero_and_inANDdecr_above_zero


[]

In [2621]:
i2= 0
r_id = flat_below_zero_and_indecr_and_stable_above_zero[0]
flat_below_zero_and_incr_and_stable_above_zero= list()
flat_below_zero_and_decr_and_stable_above_zero= list()

for r_id in flat_below_zero_and_indecr_and_stable_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = flat_below_zero_and_indecr_and_stable_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            flat_below_zero_and_decr_and_stable_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            flat_below_zero_and_incr_and_stable_above_zero.append(r_id)

    
    i2= i2 + 1


set1= set(flat_below_zero_and_indecr_and_stable_above_zero)
set2= set(flat_below_zero_and_decr_and_stable_above_zero)
set3= set(flat_below_zero_and_incr_and_stable_above_zero)
flat_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
flat_below_zero_and_inANDdecr_above_zero


[]

In [2622]:
flat_below_zero_and_decr_and_stable_above_zero

['EX_h_e',
 'EX_enter_e',
 'ACKr',
 'ALATA_D2',
 'ALATA_L2',
 'ADNK1',
 'ADNUC',
 'ADPT',
 'AMPN',
 'ASPt2pp_copy2',
 'DHAPT',
 'FA120ACPHi',
 'FA140ACPHi',
 'FA141ACPHi',
 'FA160ACPHi',
 'FA161ACPHi',
 'FDH4pp',
 'ASNN',
 'FDH5pp',
 'FACOAL60t2pp',
 'ASNNpp',
 'FTHFLi',
 'GLUN',
 'GART',
 'GDPMNP',
 'GLUSy',
 'HXCT',
 'HXPRT',
 'GLYCTO2',
 'GLYCTO3',
 'HEX1',
 'HEX7',
 'GLYCTO4',
 'INSH',
 'INSK',
 'Kt3pp',
 'MN6PP',
 'GSNK',
 'NTD7',
 'NTD9',
 'NTPTP2',
 'PFK',
 'PFK_3',
 'GUAPRT',
 'PDH',
 'PRPPS',
 'R5PP',
 'UPPRT',
 'URIH',
 'URIK2',
 'PPM',
 'PYRDC']

In [2623]:
i2= 0
r_id = indecr_and_stable_below_zero_and_flat_above_zero[0]
incr_and_stable_below_zero_and_flat_above_zero= list()
decr_and_stable_below_zero_and_flat_above_zero= list()

for r_id in indecr_and_stable_below_zero_and_flat_above_zero:                    
    
    i=0
    r_id = indecr_and_stable_below_zero_and_flat_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            decr_and_stable_below_zero_and_flat_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_and_stable_below_zero_and_flat_above_zero.append(r_id)

    
    i2= i2 + 1


set1= set(indecr_and_stable_below_zero_and_flat_above_zero)
set2= set(incr_and_stable_below_zero_and_flat_above_zero)
set3= set(decr_and_stable_below_zero_and_flat_above_zero)
inANDdecr_and_stable_below_zero_and_flat_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_flat_above_zero


[]

In [2624]:
incr_and_stable_below_zero_and_flat_above_zero

['CRNDt2rpp', 'FRD2', 'Htex', 'NADH17pp', 'NADH18pp', 'PTAr']

In [2625]:
decr_and_stable_below_zero_and_flat_above_zero

[]

In [2626]:
i2= 0
r_id = diff_indecr_both_positive_and_negative[0]
incr_both_positive_and_negative= list()
decr_both_positive_and_negative= list()
one_to_three_data_point_both_positive_and_negative= list()

for r_id in diff_indecr_both_positive_and_negative:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = diff_indecr_both_positive_and_negative[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>3:
        
        if my_series.is_monotonic_decreasing ==True:
            decr_both_positive_and_negative.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_both_positive_and_negative.append(r_id)
    
    elif len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4:
        
        one_to_three_data_point_both_positive_and_negative.append(r_id)

    
    i2= i2 + 1


set1= set(diff_indecr_both_positive_and_negative)
set2= set(decr_both_positive_and_negative)
set3= set(incr_both_positive_and_negative)
inANDdecr_both_positive_and_negative= list(set1-set2-set3)
inANDdecr_both_positive_and_negative


['FEOXAMR2',
 'FACOAE181',
 'TRPS3',
 'AGM3PH',
 'EX_quin_e',
 'ENTERES',
 'EX_ura_e',
 'PGPP180',
 'G3PGabcpp',
 'LPLIPAL2E160',
 'ENTERES2',
 'RNTR1c',
 'EX_indole_e',
 '2AGPGAT141',
 'FACOAE140',
 'LPLIPAL2E120',
 'UM3PL',
 'AGM3Pt2pp',
 'ARBTNR2',
 'PGPP181',
 'PGPP120',
 'MLDCP2Bpp',
 'EX_val__L_e',
 'FEENTERR2',
 '2AGPEAT181',
 'CPGNR2',
 'FACOAE160',
 'CLPNH160pp',
 '2AGPGAT180',
 'PGPP140',
 'LPLIPAL2G161',
 'AGM3PA',
 'FACOAE141',
 'GPDDA4pp',
 'EX_co2_e',
 'PUNP2',
 'PAPSR',
 'LPLIPAL2G160',
 'CLPNH120pp',
 'G3PEabcpp',
 '4PCPpp',
 'AGM3PApp',
 '2AGPEAT160',
 'AGM4PCP',
 'MLDEP2pp',
 'DCTPD',
 'GMPR',
 'HXAND',
 'MLTGY1pp',
 '2AGPEAT120',
 'PA120abcpp',
 'AGMt2pp',
 'GARFT',
 'LPLIPAL2E140',
 'LPLIPAL2G181',
 'CLPNH180pp',
 '2AGPEAT140',
 'CBMKr',
 'LPLIPAL2E141',
 '2AGPGAT181',
 'FACOAE180',
 'CLPNH140pp',
 'LPLIPAL2G180',
 '2AGPGAT161',
 'PA180abcpp',
 'RNTR3c',
 'PA161abcpp',
 '2AGPEAT161',
 'CLPNH141pp',
 'LPLIPAL2E181',
 'PA141abcpp',
 'PA181abcpp',
 'PA140abcpp',
 'GTHR

In [2627]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(inANDdecr_both_positive_and_negative):
    
    i=0
    r_id = inANDdecr_both_positive_and_negative[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 301:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 300
    l= list()
    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1

w5


[]

In [2628]:
w6

['EX_ura_e', 'EX_indole_e']

In [2629]:
inANDdecr_both_positive_and_negative.remove("EX_ura_e")
inANDdecr_both_positive_and_negative.remove("EX_indole_e")

increasing_below_zero_and_decreasing_above_zero= ["EX_ura_e", "EX_indole_e"]


In [2630]:
i=0
A= list()
b= list()
c= list()
d= list()
e= list()
decr_below_zero_and_decr_and_stable_above_zero=list()
decr_below_zero_and_incr_and_stable_above_zero= list()
incr_below_zero_and_incr_and_stable_above_zero= list()
incr_below_zero_and_decr_and_stable_above_zero= list()
inANDdecr_below_zero_and_decr_and_stable_above_zero= list()
inANDdecr_below_zero_and_incr_and_stable_above_zero= list()
inANDdecr_below_zero_and_decr_and_stable_above_zero=list()
inANDdecr_below_zero_and_incr_and_stable_above_zero=list()
decr_below_zero_and_inANDdecr_and_stable_above_zero=list()
incr_below_zero_and_inANDdecr_and_stable_above_zero=list()
rest= list()
rest2= list()

while i < len(diff_indecr_below_zero_and_indecr_and_stable_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i]))
    n= 0
    l= list()
    
    while n < 301:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    boolean = csvFile2["Biomass"].duplicated().any()
    
    
    n2= 300
    l2= list()
    
    while n2 < 600:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    boolean2 = csvFile3["Biomass"].duplicated().any()
    
    if len(csvFile2['Biomass'])>3:
        
        if csvFile2["Biomass"].is_monotonic_decreasing ==True and csvFile3["Biomass"].is_monotonic_decreasing ==True and r_id not in decr_below_zero_and_decr_and_stable_above_zero:
            decr_below_zero_and_decr_and_stable_above_zero.append(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i])
        elif csvFile2["Biomass"].is_monotonic_decreasing ==True and csvFile3["Biomass"].is_monotonic ==True and r_id not in decr_below_zero_and_incr_and_stable_above_zero:
            decr_below_zero_and_incr_and_stable_above_zero.append(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i])
        elif csvFile2["Biomass"].is_monotonic ==True and csvFile3["Biomass"].is_monotonic ==True and r_id not in incr_below_zero_and_incr_and_stable_above_zero:
            incr_below_zero_and_incr_and_stable_above_zero.append(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i])
        elif csvFile2["Biomass"].is_monotonic ==True and csvFile3["Biomass"].is_monotonic_decreasing ==True and r_id not in incr_below_zero_and_decr_and_stable_above_zero:
            incr_below_zero_and_decr_and_stable_above_zero.append(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i])
        elif csvFile2["Biomass"].is_monotonic ==False and csvFile2["Biomass"].is_monotonic_decreasing ==False and csvFile3["Biomass"].is_monotonic_decreasing ==True and r_id not in inANDdecr_below_zero_and_decr_and_stable_above_zero:
            inANDdecr_below_zero_and_decr_and_stable_above_zero.append(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i])
        elif csvFile2["Biomass"].is_monotonic ==False and csvFile2["Biomass"].is_monotonic_decreasing ==False and csvFile3["Biomass"].is_monotonic ==True and r_id not in inANDdecr_below_zero_and_incr_and_stable_above_zero:
            inANDdecr_below_zero_and_incr_and_stable_above_zero.append(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i])
        elif csvFile2["Biomass"].is_monotonic_decreasing ==True and csvFile3["Biomass"].is_monotonic ==False and csvFile3["Biomass"].is_monotonic_decreasing ==False and r_id not in decr_below_zero_and_inANDdecr_and_stable_above_zero:
            decr_below_zero_and_inANDdecr_and_stable_above_zero.append(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i])
        elif csvFile2["Biomass"].is_monotonic ==True and csvFile3["Biomass"].is_monotonic ==False and csvFile3["Biomass"].is_monotonic_decreasing ==False and r_id not in incr_below_zero_and_inANDdecr_and_stable_above_zero:
            incr_below_zero_and_inANDdecr_and_stable_above_zero.append(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i])

    else:
        rest2.append(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i])
    
    i=i+1
    
incr_below_zero_and_decr_and_stable_above_zero


['EX_feenter_e',
 'AACPS5',
 'AACPS6',
 'AACPS7',
 'AACPS1',
 'AACPS2',
 'AACPS3',
 'AACPS4',
 'AKGDH',
 'CDAPPA181',
 'APH120',
 'APH140',
 'APH141',
 'APH160',
 'APH161',
 'APH180',
 'CDAPPA120',
 'APH181',
 'CDAPPA140',
 'CDAPPA141',
 'CDAPPA160',
 'CDAPPA161',
 'CDAPPA180',
 'CPPPGO',
 'DADK',
 'CYSabcpp',
 'DGK1',
 'CYTDK2',
 'CYTK1',
 'CYTK2',
 'ECOAH4',
 'ECOAH5',
 'ECOAH6',
 'ECOAH7',
 'DURIK1',
 'F6PA',
 'FA100ACPHi',
 'FA80ACPHi',
 'FTHFD',
 'G3PT',
 'G6PDA',
 'GLUDC',
 'GLYAT',
 'ICL',
 'GND',
 'GNK',
 'NDPK3',
 'NDPK5',
 'NDPK7',
 'NDPK8',
 'NTD1',
 'NTD11',
 'NTD2',
 'P5CD',
 'PGI',
 'PGL',
 'PROD2',
 'PIt2rpp',
 'PUNP3',
 'RPE',
 'SPMDabcpp',
 'TKT1',
 'TKT2',
 'TMDK1',
 'PPC',
 'PPPGO3']

In [2631]:
decr_below_zero_and_decr_and_stable_above_zero

['ALAALAD',
 'ALAabcpp',
 'ALDD2x',
 'ASPabcpp',
 'CLIPAabctex',
 'CD2abcpp',
 'COBALT2abcpp',
 'CRNDabcpp',
 'CRNabcpp',
 'CTBTabcpp',
 'ARBabcpp',
 'ARGabcpp',
 'FE2abcpp',
 'ASNabcpp',
 'G3PD5',
 'G3PD6',
 'G3PD7',
 'GLCabcpp',
 'GLUabcpp',
 'ILEabcpp',
 'HISabcpp',
 'Kabcpp',
 'LDH_D2',
 'LEUabcpp',
 'LIPAabctex',
 'LYSabcpp',
 'MDH2',
 'MDH3',
 'NADH10',
 'NADH5',
 'NADH9',
 'NADPHQR2',
 'NADPHQR3',
 'NADPHQR4',
 'NI2abcpp',
 'NI2uabcpp',
 'NTP1',
 'NTP10',
 'NTP3',
 'NTP5',
 'NTPP1',
 'NTPP2',
 'NTPP3',
 'NTPP4',
 'NTPP5',
 'NTPP6',
 'NTPP7',
 'NTPP8',
 'ORNabcpp',
 'PROabcpp',
 'PIuabcpp',
 'PTRCabcpp',
 'THRabcpp',
 'VALabcpp',
 'ZN2abcpp',
 'ZNabcpp',
 'PPA',
 'PPA2',
 'PPPGO',
 'DARBabcpp']

In [2632]:
incr_below_zero_and_incr_and_stable_above_zero

['EX_h2o_e', 'SUCDi']

In [2633]:
incr_below_zero_and_inANDdecr_and_stable_above_zero

['ECOAH1',
 'ECOAH2',
 'ECOAH3',
 'ENO',
 'FUM',
 'GHMT2r',
 'GAPD',
 'MDH',
 'MTHFC',
 'MTHFD',
 'TPI']

In [2634]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(incr_below_zero_and_inANDdecr_and_stable_above_zero):
    
    i=0
    r_id = incr_below_zero_and_inANDdecr_and_stable_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 301:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 300
    l= list()
    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1

w5


[]

In [2635]:
w7

incr_below_zero_and_inANDdecr_and_stable_above_zero.remove("ECOAH1")
incr_below_zero_and_decr_and_stable_above_zero.append("ECOAH1")
incr_below_zero_and_inANDdecr_and_stable_above_zero.remove("ECOAH2")
incr_below_zero_and_decr_and_stable_above_zero.append("ECOAH2")
incr_below_zero_and_inANDdecr_and_stable_above_zero.remove("ECOAH3")
incr_below_zero_and_decr_and_stable_above_zero.append("ECOAH3")


In [2636]:
i=0
A= list()
b= list()
c= list()
d= list()
e= list()
decreasing_and_stable_below_zero_and_decreasing_above_zero= list()
decreasing_and_stable_below_zero_and_decreasing_and_stable_above_zero= list()


while i < len(mon_decr_and_stable_both_positive_and_negative):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(mon_decr_and_stable_both_positive_and_negative[i]))
    n= 0
    l= list()
    
    while n < 301:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    boolean = csvFile2["Biomass"].duplicated().any()
    
    
    n2= 300
    l2= list()
    
    while n2 < 600:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    boolean2 = csvFile3["Biomass"].duplicated().any()
    
    if boolean == True and boolean2 == False:
        A.append(mon_decr_and_stable_both_positive_and_negative[i])
        
        if len(csvFile2['Biomass'].unique()) > 1:
            decreasing_and_stable_below_zero_and_decreasing_above_zero.append(mon_decr_and_stable_both_positive_and_negative[i])
        
        if len(csvFile2['Biomass'].unique()) == 1:
            flat_below_zero_and_mon_decr_above_zero.append(mon_decr_and_stable_both_positive_and_negative[i])
        
    
    if boolean == False and boolean2 == False:
        b.append(mon_decr_and_stable_both_positive_and_negative[i])
    
    if boolean == False and boolean2 == True:
        c.append(mon_decr_and_stable_both_positive_and_negative[i])

    if boolean == True:
        d.append(mon_decr_and_stable_both_positive_and_negative[i])
    
    if boolean2 == True:
        e.append(mon_decr_and_stable_both_positive_and_negative[i])
    
    if boolean == True and boolean2 == True:
        
        if len(csvFile2['Biomass'].unique()) > 1 and len(csvFile3['Biomass'].unique()) > 1:
            decreasing_and_stable_below_zero_and_decreasing_and_stable_above_zero.append(mon_decr_and_stable_both_positive_and_negative[i])
        
        if len(csvFile2['Biomass'].unique()) == 1 and len(csvFile3['Biomass'].unique()) > 1:
            flat_below_zero_and_decreasing_and_stable_above_zero.append(mon_decr_and_stable_both_positive_and_negative[i])
        
        if len(csvFile2['Biomass'].unique()) > 1 and len(csvFile3['Biomass'].unique()) == 1:
            decreasing_and_stable_below_zero_and_flat_above_zero.append(mon_decr_and_stable_both_positive_and_negative[i])

    i=i+1

decreasing_and_stable_below_zero_and_decreasing_above_zero


[]

In [2637]:
decreasing_and_stable_below_zero_and_decreasing_and_stable_above_zero

['ALDD2y',
 'ASPt2_2pp',
 'ASPt2_3pp',
 'CMtpp',
 'CYTBD2pp',
 'CYTBDpp',
 'DAAD',
 'ARBt3ipp',
 'DOXRBCNtpp',
 'F6PP',
 'FADRx2',
 'FE2t2pp',
 'FUSAtpp',
 'G6PP',
 'ASPT',
 'GUAt2pp',
 'GLYCLTDy',
 'IDOND2',
 'HPYRRy',
 'LCTSt3ipp',
 'MALt2_3pp',
 'MINCYCtpp',
 'ME1',
 'MLTG3',
 'MLTG4',
 'MLTG5',
 'MNt2pp',
 'MG2tpp',
 'NADTRHD',
 'NAt3_1p5pp',
 'NAt3_2pp',
 'NOVBCNtpp',
 'OAADC',
 'RFAMPtpp',
 'SSALx',
 'SUCCt2_3pp',
 'THRA2',
 'TTRCYCtpp',
 'XANt2pp']

In [2638]:
z=0
f= list()
g= list()
h= list()
i= list()
j= list()
flat_below_zero_and_increasing_above_zero= list()
increasing_and_stable_below_zero_and_decreasing_above_zero= list()
increasing_and_stable_below_zero_and_increasing_and_stable_above_zero= list()
increasing_below_zero_and_flat_above_zero= list()
flat_below_zero_and_increasing_and_stable_above_zero= list()
mon_incr_and_stable_below_zero_and_flat_above_zero= list()

while z < len(mon_incr_and_stable_both_positive_and_negative):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(mon_incr_and_stable_both_positive_and_negative[z]))
    
    n= 0
    l= list()
    
    while n < 301:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    boolean = csvFile2["Biomass"].duplicated().any()
    
    
    n2= 300
    l2= list()
    
    while n2 < 600:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1
    
    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    boolean2 = csvFile3["Biomass"].duplicated().any()
    
    if boolean == True and boolean2 == False:
        f.append(mon_incr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile2['Biomass'].unique()) > 1:
            increasing_and_stable_below_zero_and_decreasing_above_zero.append(mon_incr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile2['Biomass'].unique()) == 1:
            flat_below_zero_and_increasing_above_zero.append(mon_incr_and_stable_both_positive_and_negative[z])
        
    
    if boolean == False and boolean2 == False:
        g.append(mon_incr_and_stable_both_positive_and_negative[z])
    
    if boolean == False and boolean2 == True:
        h.append(mon_incr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile3['Biomass'].unique()) == 1:
            
            increasing_below_zero_and_flat_above_zero.append(mon_incr_and_stable_both_positive_and_negative[z])

    if boolean == True:
        i.append(mon_incr_and_stable_both_positive_and_negative[z])
    
    if boolean2 == True:
        j.append(mon_incr_and_stable_both_positive_and_negative[z])
    
    if boolean == True and boolean2 == True:
        
        if len(csvFile2['Biomass'].unique()) > 1 and len(csvFile3['Biomass'].unique()) > 1:
            increasing_and_stable_below_zero_and_increasing_and_stable_above_zero.append(mon_incr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile2['Biomass'].unique()) == 1 and len(csvFile3['Biomass'].unique()) > 1:
            flat_below_zero_and_increasing_and_stable_above_zero.append(mon_incr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile2['Biomass'].unique()) > 1 and len(csvFile3['Biomass'].unique()) == 1:
            mon_incr_and_stable_below_zero_and_flat_above_zero.append(mon_incr_and_stable_both_positive_and_negative[z])

    z=z+1

increasing_and_stable_below_zero_and_decreasing_above_zero


[]

In [2639]:
z=0
k= list()
L= list()
m= list()
o= list()
p= list()
increasingORANDdecreasing_and_stable_below_zero_and_increasingORANDdecreasing_above_zero= list()
flat_below_zero_and_increasingANDdecreasing_above_zero= list()
inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list()
decreasing_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list()
increasing_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list()
inANDdecr_and_stable_below_zero_and_decreasing_and_stable_above_zero= list()
inANDdecr_and_stable_below_zero_and_increasing_and_stable_above_zero= list()
flat_below_zero_and_inANDdecr_and_stable_above_zero= list()
inANDdecr_and_stable_below_zero_and_flat_above_zero= list()
decreasing_and_stable_below_zero_and_increasing_and_stable_above_zero= list()

while z < len(inANDdecr_and_stable_both_positive_and_negative):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(inANDdecr_and_stable_both_positive_and_negative[z]))
    
    n= 0
    l= list()
    
    while n < 301:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    boolean = csvFile2["Biomass"].duplicated().any()
    
    
    n2= 300
    l2= list()
    
    while n2 < 600:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    boolean2 = csvFile3["Biomass"].duplicated().any()
    
    if boolean == True and boolean2 == False:
        k.append(inANDdecr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile2['Biomass'].unique()) > 1:
            increasingORANDdecreasing_and_stable_below_zero_and_increasingORANDdecreasing_above_zero.append(inANDdecr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile2['Biomass'].unique()) == 1:
            flat_below_zero_and_increasingANDdecreasing_above_zero.append(inANDdecr_and_stable_both_positive_and_negative[z])
        
    
    if boolean == False and boolean2 == False:
        L.append(inANDdecr_and_stable_both_positive_and_negative[z])
    
    if boolean == False and boolean2 == True:
        m.append(inANDdecr_and_stable_both_positive_and_negative[z])

    if boolean == True:
        o.append(inANDdecr_and_stable_both_positive_and_negative[z])
    
    if boolean2 == True:
        p.append(inANDdecr_and_stable_both_positive_and_negative[z])
    
    if boolean == True and boolean2 == True:
        
        if len(csvFile2['Biomass'].unique()) > 1 and len(csvFile3['Biomass'].unique()) > 1:
            inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero.append(inANDdecr_and_stable_both_positive_and_negative[z])
                
        if len(csvFile2['Biomass'].unique()) == 1 and len(csvFile3['Biomass'].unique()) > 1:
            flat_below_zero_and_inANDdecr_and_stable_above_zero.append(inANDdecr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile2['Biomass'].unique()) > 1 and len(csvFile3['Biomass'].unique()) == 1:
            inANDdecr_and_stable_below_zero_and_flat_above_zero.append(inANDdecr_and_stable_both_positive_and_negative[z])

        
    z=z+1

increasingORANDdecreasing_and_stable_below_zero_and_increasingORANDdecreasing_above_zero


['RPI']

In [2640]:
m

incr_below_zero_and_decr_and_stable_above_zero.append("FE2t3pp")


In [2641]:
inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero


['ZN2t3pp',
 'FORt2pp',
 'NMNAT',
 'PFL',
 'FADRx',
 'TRE6PH',
 'PPS',
 'THD2pp',
 'FBP',
 'SUCMALtpp',
 'FBA',
 'Kt2pp',
 'ZN2tpp',
 'GLCt2pp',
 'CYStpp',
 'CITL',
 'ASNS2',
 'COBALT2t3pp',
 'MALS',
 'ACALD',
 'NI2t3pp',
 'PUNP5',
 'GDPMNH',
 'PYNP2r',
 'NI2tpp',
 'THRt4pp',
 'GDPDPK',
 'ME2',
 'PPCK',
 'MALt2_2pp',
 'SSALy',
 'GLYK',
 'PYK',
 'ARBt2rpp',
 'PTRCt2pp',
 'CD2t3pp',
 'MACPD',
 'FE2tpp',
 'SUCASPtpp',
 'SUCCt2_2pp',
 'GLYCDx',
 'GLYCLTDx',
 'PPK2',
 'XYLI2',
 'GDPTPDP',
 'LYSt2pp',
 'FLDR2',
 'TALA',
 'PUNP1',
 'MN2tpp']

In [2642]:
inANDdecr_and_stable_below_zero_and_decreasing_above_zero= list(increasingORANDdecreasing_and_stable_below_zero_and_increasingORANDdecreasing_above_zero)
inANDdecr_and_stable_below_zero_and_decreasing_above_zero


['RPI']

In [2643]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero):
    
    i=0
    r_id = inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 301:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 300
    l= list()
    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1

w5


[]

In [2644]:
i2=0
decreasing_and_stable_below_zero_and_increasing_and_stable_above_zero= list()
increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero= list()
decreasing_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list()
increasing_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list()
inANDdecr_and_stable_below_zero_and_decreasing_and_stable_above_zero= list()
inANDdecr_and_stable_below_zero_and_increasing_and_stable_above_zero= list()
rest2= list()

while i2 < len(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero):
    
    i=0
    r_id = inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 301:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)
    
    i= 300
    l2= list()
    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l2.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)

    my_series2 = csvFile3['Biomass'].squeeze()
    type(my_series2)

    if len(csvFile2['Biomass'])>3 and len(csvFile3['Biomass'])>3:
        
        if my_series.is_monotonic_decreasing ==True and my_series2.is_monotonic_decreasing ==True:
            decreasing_and_stable_below_zero_and_decreasing_and_stable_above_zero.append(r_id)
        elif my_series.is_monotonic ==True and my_series2.is_monotonic ==True:
            increasing_and_stable_below_zero_and_increasing_and_stable_above_zero.append(r_id)
        elif my_series.is_monotonic_decreasing ==True and my_series2.is_monotonic ==True:
            decreasing_and_stable_below_zero_and_increasing_and_stable_above_zero.append(r_id)
        elif my_series.is_monotonic ==True and my_series2.is_monotonic_decreasing ==True:
            increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero.append(r_id)
        elif my_series.is_monotonic_decreasing ==True and my_series2.is_monotonic_decreasing ==False and my_series2.is_monotonic ==False:
            decreasing_and_stable_below_zero_and_inANDdecr_and_stable_above_zero.append(r_id)
        elif my_series.is_monotonic ==True and my_series2.is_monotonic_decreasing ==False and my_series2.is_monotonic ==False:
            increasing_and_stable_below_zero_and_inANDdecr_and_stable_above_zero.append(r_id)
        elif my_series.is_monotonic_decreasing ==False and my_series.is_monotonic ==False and my_series2.is_monotonic_decreasing ==True:
            inANDdecr_and_stable_below_zero_and_decreasing_and_stable_above_zero.append(r_id)
        elif my_series.is_monotonic_decreasing ==False and my_series.is_monotonic ==False and my_series2.is_monotonic ==True:
            inANDdecr_and_stable_below_zero_and_increasing_and_stable_above_zero.append(r_id)
            
    else:
        
        rest2.append(r_id)

    
    i2= i2 + 1

decreasing_and_stable_below_zero_and_increasing_and_stable_above_zero


[]

In [2645]:
inANDdecr_and_stable_below_zero_and_decreasing_and_stable_above_zero

['PPS', 'THD2pp', 'FBP', 'CITL', 'ME2', 'PPCK', 'GLYCDx', 'FLDR2']

In [2646]:
increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero

['ZN2t3pp',
 'FORt2pp',
 'NMNAT',
 'PFL',
 'FADRx',
 'TRE6PH',
 'SUCMALtpp',
 'FBA',
 'Kt2pp',
 'ZN2tpp',
 'GLCt2pp',
 'CYStpp',
 'ASNS2',
 'COBALT2t3pp',
 'MALS',
 'ACALD',
 'NI2t3pp',
 'PUNP5',
 'GDPMNH',
 'PYNP2r',
 'NI2tpp',
 'THRt4pp',
 'GDPDPK',
 'MALt2_2pp',
 'SSALy',
 'GLYK',
 'PYK',
 'ARBt2rpp',
 'PTRCt2pp',
 'CD2t3pp',
 'MACPD',
 'FE2tpp',
 'SUCASPtpp',
 'SUCCt2_2pp',
 'GLYCLTDx',
 'PPK2',
 'XYLI2',
 'GDPTPDP',
 'LYSt2pp',
 'TALA',
 'PUNP1',
 'MN2tpp']

In [2647]:
i2= 0
r_id = indecr_and_stable_below_zero_and_mon_decr_above_zero[0]
incr_and_stable_below_zero_and_mon_decr_above_zero= list()
decr_and_stable_below_zero_and_mon_decr_above_zero= list()
one_to_three_data_point_below_zero_and_mon_decr_above_zero= list()

for r_id in indecr_and_stable_below_zero_and_mon_decr_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = indecr_and_stable_below_zero_and_mon_decr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 301:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>3:
        
        if my_series.is_monotonic_decreasing ==True:
            decr_and_stable_below_zero_and_mon_decr_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_and_stable_below_zero_and_mon_decr_above_zero.append(r_id)
    
    elif len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4:
        
        one_to_three_data_point_below_zero_and_mon_decr_above_zero.append(r_id)

    
    i2= i2 + 1

decr_and_stable_below_zero_and_mon_decr_above_zero

set1= set(indecr_and_stable_below_zero_and_mon_decr_above_zero)
set2= set(decr_and_stable_below_zero_and_mon_decr_above_zero)
set3= set(incr_and_stable_below_zero_and_mon_decr_above_zero)
set4= set(one_to_three_data_point_below_zero_and_mon_decr_above_zero)
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero= list(set1-set2-set3-set4)
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero


['ACOAD2f', 'ACOAD3f', 'GLUDy', 'ACOAD1f', 'PGK', 'CO2tpp', 'PGM']

In [2648]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero):
    
    i=0
    r_id = inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 301:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 300
    l= list()
    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1

w5


[]

In [2649]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero):
    
    i=0
    r_id = inANDdecr_and_stable_below_zero_and_mon_decr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 301:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 300
    l= list()
    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1

w5


[]

In [2650]:
w6

inANDdecr_and_stable_below_zero_and_mon_decr_above_zero.remove("ACOAD2f")
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero.remove("ACOAD3f")
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero.remove("ACOAD1f")
incr_and_stable_below_zero_and_mon_decr_above_zero.append("ACOAD2f")
incr_and_stable_below_zero_and_mon_decr_above_zero.append("ACOAD3f")
incr_and_stable_below_zero_and_mon_decr_above_zero.append("ACOAD1f")


In [2651]:
i2= 0
r_id = diff_indecr_above_zero[0]
incr_above_zero= list()
decr_above_zero= list()
one_to_three_data_point_above_zero= list()

for r_id in diff_indecr_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = diff_indecr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>3:
        
        if my_series.is_monotonic_decreasing ==True:
            decr_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_above_zero.append(r_id)
    
    elif len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4:
        
        one_to_three_data_point_above_zero.append(r_id)

    
    i2= i2 + 1

decr_above_zero


['DM_oxam_c',
 'EX_12ppd__R_e',
 'EX_15dap_e',
 'EX_h2s_e',
 'EX_3hpp_e',
 'EX_his__L_e',
 'EX_hom__L_e',
 'EX_5mtr_e',
 'EX_kdo2lipid4_e',
 'EX_acser_e',
 'EX_lipa_e',
 'EX_lipa_cold_e',
 'EX_cit_e',
 'EX_pheme_e',
 '2AGPA120tipp',
 '2AGPA140tipp',
 '2AGPA141tipp',
 '2AGPA160tipp',
 '2AGPA161tipp',
 '2AGPA180tipp',
 '2AGPA181tipp',
 '2AGPE120tipp',
 '2AGPE140tipp',
 '2AGPE141tipp',
 'EX_spmd_e',
 '2AGPE160tipp',
 '2AGPE161tipp',
 '2AGPE180tipp',
 '2AGPE181tipp',
 'EX_thym_e',
 '2AGPG120tipp',
 '2AGPG140tipp',
 '2AGPG141tipp',
 '2AGPG160tipp',
 '2AGPG161tipp',
 '2AGPG180tipp',
 '2AGPG181tipp',
 'EX_urea_e',
 '3HAD100',
 '3AMACHYD',
 '3HAD120',
 '3HAD121',
 '3HAD140',
 '3HAD141',
 '3HAD160',
 '3HAD161',
 '3HAD180',
 '3HAD181',
 '3HAD40',
 '3HAD60',
 '3HAD80',
 'AACTOOR',
 '3HPPtpp',
 'ABUTD',
 '3OAR100',
 '3OAR120',
 '3OAR121',
 '3OAR141',
 '3OAR160',
 '3OAR161',
 '3OAR180',
 '3OAR181',
 '3OAR40',
 '3OAR60',
 '3OAR80',
 '3OAS100',
 '3OAS120',
 '3OAS121',
 '3OAS141',
 '3OAS160',
 '3OAS16

In [2652]:
one_to_three_data_point_above_zero

['Ktex']

In [2653]:
incr_above_zero

['BIOMASS_Ec_iJO1366_core_53p95M', 'SPODM']

In [2654]:
set1= set(diff_indecr_above_zero)
set2= set(decr_above_zero)
set3= set(incr_above_zero)
set4= set(one_to_three_data_point_above_zero)
inANDdecr_above_zero= list(set1-set2-set3-set4)
inANDdecr_above_zero


['AICART',
 'PE160abcpp',
 'DHAD1',
 'PPBNGS',
 'USHD',
 'SHSL1',
 'PPND',
 'HSTPT',
 'ARGSL',
 'O2tpp',
 'IG3PS',
 'G1SAT',
 'UAGCVT',
 'PRAGSr',
 'K2L4Aabctex',
 'GLUTRR',
 'GMPS2',
 'NH4tex',
 'ADSL2r',
 'ATPPRT',
 'DAPDC',
 'LEUTAi',
 'PE161abcpp',
 'UAMAGS',
 'THRS',
 'HSK',
 'G5SADs',
 'CHORS',
 'THRD_L',
 'MPTG',
 'ANPRT',
 'PSD160',
 'DHAD2',
 'DM_5drib_c',
 'SERAT',
 'DAPE',
 'GLUTRS',
 'UGMDDS',
 'ASPCT',
 'SADT2',
 'KDOPP',
 'IGPS',
 'CYSS',
 'PRASCSi',
 'THDPS',
 'PRATPP',
 'UAAGDS',
 'ACGK',
 '5DOAN',
 'KDOPS',
 'ARGSS',
 'UPPDC1',
 'DHQTi',
 'U23GAAT',
 'IPPS',
 'SHK3Dr',
 'DHDPRy',
 'MTHFR2',
 'ASPK',
 'APG3PAT160',
 'PSCVT',
 'HMBS',
 'AIRC2',
 'OMPDC',
 'K2L4Aabcpp',
 'TMDS',
 '3OAR140',
 'NADS1',
 'DHQS',
 'CHORM',
 'BPNT',
 'UPP3S',
 'PRAIS',
 'UAGDP',
 'PSD161',
 'G1PACT',
 'METAT',
 'TDSK',
 '3OAS140',
 'HISTD',
 'PRMICI',
 'DDPA',
 'PAPPT3',
 'HISTP',
 'AGPAT160',
 'UDCPDP',
 'KDOCT2',
 'ACHBS',
 'OCBT',
 'SO4tex',
 'UHGADA',
 'MOAT',
 'SDPDS',
 'UAPGR',
 'IGPDH',

In [2655]:
w7= list()
w8= list()
i2=0

while i2 < len(inANDdecr_above_zero):
    
    i=0
    r_id = inANDdecr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1
    
w7


['PE160abcpp',
 'PPBNGS',
 'USHD',
 'SHSL1',
 'PPND',
 'HSTPT',
 'ARGSL',
 'IG3PS',
 'G1SAT',
 'UAGCVT',
 'K2L4Aabctex',
 'GLUTRR',
 'GMPS2',
 'ATPPRT',
 'PE161abcpp',
 'UAMAGS',
 'G5SADs',
 'THRD_L',
 'MPTG',
 'ANPRT',
 'PSD160',
 'DHAD2',
 'DM_5drib_c',
 'SERAT',
 'GLUTRS',
 'UGMDDS',
 'SADT2',
 'KDOPP',
 'IGPS',
 'CYSS',
 'PRATPP',
 'UAAGDS',
 'ACGK',
 '5DOAN',
 'KDOPS',
 'ARGSS',
 'UPPDC1',
 'U23GAAT',
 'MTHFR2',
 'APG3PAT160',
 'HMBS',
 'K2L4Aabcpp',
 'TMDS',
 '3OAR140',
 'NADS1',
 'BPNT',
 'UPP3S',
 'UAGDP',
 'PSD161',
 'G1PACT',
 'METAT',
 'TDSK',
 '3OAS140',
 'HISTD',
 'PRMICI',
 'PAPPT3',
 'HISTP',
 'AGPAT160',
 'UDCPDP',
 'KDOCT2',
 'ACHBS',
 'OCBT',
 'SO4tex',
 'UHGADA',
 'MOAT',
 'UAPGR',
 'IGPDH',
 'PRAMPC',
 'UAGPT3',
 'MCTP1App',
 'PSSA160',
 'ADSS',
 'CTPS2',
 'FCLT',
 'NNATr',
 'ANS',
 'UAMAS',
 'METS',
 'MOAT2',
 'ADSL1r',
 'LPADSS',
 'DHFR',
 'CYSTL',
 'ACGS',
 'HSST',
 'PPNDH',
 'A5PISO',
 'KARA2',
 'SULR',
 'AGPAT161',
 'APG3PAT161',
 'PRAIi',
 'ADSK',
 'ACODA',
 '

In [2656]:
set1= set(inANDdecr_above_zero)
set2= set(w7)
set3= set1-set2
inANDdecr_above_zero= list(set3)

decr_above_zero= list(decr_above_zero) + list(w7)


In [2657]:
i2= 0
r_id = diff_indecr_below_zero[0]
incr_below_zero= list()
decr_below_zero= list()
one_to_three_data_point_below_zero= list()

for r_id in diff_indecr_below_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = diff_indecr_below_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>3:
        
        if my_series.is_monotonic_decreasing ==True:
            decr_below_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_below_zero.append(r_id)
    
    elif len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4:
        
        one_to_three_data_point_below_zero.append(r_id)

    
    i2= i2 + 1

decr_below_zero


['EX_cmp_e',
 'EX_glc__D_e',
 'EX_glcr_e',
 'EX_glcur_e',
 'EX_glcur1p_e',
 'EX_gln__L_e',
 'EX_glyc2p_e',
 'EX_gmp_e',
 'EX_gsn_e',
 'EX_h2_e',
 'EX_23camp_e',
 'EX_23ccmp_e',
 'EX_23cgmp_e',
 'EX_23cump_e',
 'EX_23dappa_e',
 'EX_2ddglcn_e',
 'EX_hdca_e',
 'EX_3amp_e',
 'EX_hdcea_e',
 'EX_3cmp_e',
 'EX_3gmp_e',
 'EX_3hcinnm_e',
 'EX_3hpppn_e',
 'EX_3ump_e',
 'EX_LalaDglu_e',
 'EX_cytd_e',
 'EX_LalaLglu_e',
 'EX_dad_2_e',
 'EX_imp_e',
 'EX_acac_e',
 'EX_damp_e',
 'EX_ins_e',
 'EX_acgam_e',
 'EX_acgam1p_e',
 'EX_acmana_e',
 'EX_acmum_e',
 'EX_acnam_e',
 'EX_lcts_e',
 'EX_dca_e',
 'EX_adn_e',
 'EX_dcmp_e',
 'EX_dcyt_e',
 'EX_lyx__L_e',
 'EX_mal__D_e',
 'EX_ddca_e',
 'EX_malt_e',
 'EX_all__D_e',
 'EX_dgmp_e',
 'EX_man_e',
 'EX_man6p_e',
 'EX_manglyc_e',
 'EX_amp_e',
 'EX_arab__L_e',
 'EX_melib_e',
 'EX_dgsn_e',
 'EX_ascb__L_e',
 'EX_but_e',
 'EX_butso3_e',
 'EX_dimp_e',
 'EX_mnl_e',
 'EX_din_e',
 'EX_chtbs_e',
 'EX_dtmp_e',
 'EX_ocdca_e',
 'EX_ocdcea_e',
 'EX_octa_e',
 'EX_dump_e',
 'EX_x

In [2658]:
one_to_three_data_point_below_zero

['EX_k_e',
 'PAPA120pp',
 'PAPA140pp',
 'PAPA141pp',
 'PAPA160pp',
 'PAPA180pp',
 'UDCPDPpp']

In [2659]:
incr_below_zero

['EX_cpgn_e',
 'EX_arbtn_fe3_e',
 'EX_no2_e',
 '12PPDRtex',
 '12PPDRtpp',
 '12PPDStex',
 '12PPDStpp',
 'EX_taur_e',
 'EX_fe3dhbzs_e',
 'EX_fe3hox_e',
 'EX_fecrm_e',
 'EX_feoxam_e',
 '3HPPtex',
 'ABUTtex',
 'ACACCT',
 'ACALDtex',
 'ACALDtpp',
 '3PEPTtex',
 'ACGAptspp',
 '4PEPTtex',
 '5DGLCNR',
 '5DGLCNt2rpp',
 'ACMANAptspp',
 '5DGLCNtex',
 '5MTRtex',
 'AKGt2rpp',
 'AKGtex',
 'ACSERtex',
 'ALAALAtex',
 'ACtex',
 'ADEt2rpp',
 'ADEtex',
 'ALAtex',
 'ALCD19',
 'ALCD2x',
 'ALDD3y',
 'ADNt2pp',
 'ALLTNt2rpp',
 'ALLTNtex',
 'ALPATE160pp',
 'ALPATG160pp',
 'AGMtex',
 'ANHGMtex',
 'ASPtex',
 'BALAt2pp',
 'CGLYtex',
 'BTNt2ipp',
 'BUTCT',
 'CITtex',
 'DADNt2pp',
 'CSNt2pp',
 'DALAtex',
 'DAPabcpp',
 'DAPtex',
 'DCYTt2pp',
 'DDCAtexi',
 'DDGLK',
 'CYStex',
 'DGSNt2pp',
 'CYTDt2pp',
 'DHAtex',
 'DHAtpp',
 'D_LACt2pp',
 'D_LACtex',
 'DINSt2pp',
 'DSERDHr',
 'DSERt2pp',
 'ECOAH8',
 'DURIt2pp',
 'ETHAtex',
 'ETOHtex',
 'ETOHtrpp',
 'ARGtex',
 'F6Pt6_2pp',
 'FACOAL80t2pp',
 'FACOAL100t2pp',
 'FESD2s',


In [2660]:
set1= set(diff_indecr_below_zero)
set2= set(decr_below_zero)
set3= set(incr_below_zero)
set4= set(one_to_three_data_point_below_zero)
inANDdecr_below_zero= list(set1-set2-set3-set4)
inANDdecr_below_zero


['ORPT',
 'EX_pi_e',
 'AGPR',
 'PHETA1',
 'EX_skm_e',
 'IPPMIb',
 'EX_26dap__M_e',
 'DHORTS',
 'EX_metsox_R__L_e',
 'ASAD',
 'EX_orot_e',
 'EX_mso3_e',
 'AIRC3',
 'EX_nh4_e',
 'KARA1',
 'EX_met__L_e',
 'HSDy',
 'EX_so4_e',
 'TYRTA',
 'ILETA',
 'ACOTA',
 'GLUR',
 'EX_sulfac_e',
 'SDPTA',
 'EX_metsox_S__L_e',
 'EX_isetac_e',
 'PGAMT',
 'IMPC',
 'IPPMIa']

In [2661]:
w9= list()
w10= list()
i2=0

while i2 < len(inANDdecr_below_zero):
    
    i=0
    r_id = inANDdecr_below_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w9.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w10.append(r_id)
    
    i2= i2+1
    
w9


[]

In [2662]:
w10

['AGPR',
 'PHETA1',
 'EX_metsox_R__L_e',
 'EX_orot_e',
 'EX_mso3_e',
 'EX_met__L_e',
 'EX_so4_e',
 'TYRTA',
 'ILETA',
 'ACOTA',
 'GLUR',
 'EX_sulfac_e',
 'EX_metsox_S__L_e',
 'EX_isetac_e',
 'PGAMT']

In [2663]:
set1= set(inANDdecr_below_zero)
set2= set(w10)
inANDdecr_below_zero= list(set1-set2)

incr_below_zero= list(incr_below_zero) + list(w10)


In [2664]:
i=0
inANDdecr_decr_below_zero_and_incr_above_zero= list()
inANDdecr_decr_below_zero_and_decr_above_zero= list()
inANDdecr_incr_below_zero_and_decr_above_zero= list()
inANDdecr_incr_below_zero_and_incr_above_zero= list()
inANDdecr_decr_below_zero_and_inANDdecr_above_zero= list()
inANDdecr_incr_below_zero_and_inANDdecr_above_zero= list()
inANDdecr_inANDdecr_below_zero_and_decr_above_zero= list()
inANDdecr_inANDdecr_below_zero_and_incr_above_zero= list()

while i < len(inANDdecr_both_positive_and_negative):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(inANDdecr_both_positive_and_negative[i]))
    n= 0
    l= list()
    
    while n < 301:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2= 300
    l2= list()
    
    while n2 < 600:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True and my_series2.is_monotonic_decreasing ==True and inANDdecr_both_positive_and_negative[i] not in inANDdecr_decr_below_zero_and_decr_above_zero:
            inANDdecr_decr_below_zero_and_decr_above_zero.append(inANDdecr_both_positive_and_negative[i])
        elif my_series.is_monotonic ==True and my_series2.is_monotonic ==True and inANDdecr_both_positive_and_negative[i] not in inANDdecr_incr_below_zero_and_incr_above_zero:
            inANDdecr_incr_below_zero_and_incr_above_zero.append(inANDdecr_both_positive_and_negative[i])
        elif my_series.is_monotonic_decreasing ==True and my_series2.is_monotonic ==True and inANDdecr_both_positive_and_negative[i] not in inANDdecr_decr_below_zero_and_incr_above_zero:
            inANDdecr_decr_below_zero_and_incr_above_zero.append(inANDdecr_both_positive_and_negative[i])
        elif my_series.is_monotonic ==True and my_series2.is_monotonic_decreasing ==True and inANDdecr_both_positive_and_negative[i] not in inANDdecr_incr_below_zero_and_decr_above_zero:
            inANDdecr_incr_below_zero_and_decr_above_zero.append(inANDdecr_both_positive_and_negative[i])
        elif my_series.is_monotonic_decreasing ==True and inANDdecr_both_positive_and_negative[i] not in inANDdecr_decr_below_zero_and_inANDdecr_above_zero:
            inANDdecr_decr_below_zero_and_inANDdecr_above_zero.append(inANDdecr_both_positive_and_negative[i])
        elif my_series.is_monotonic ==True and inANDdecr_both_positive_and_negative[i] not in inANDdecr_incr_below_zero_and_inANDdecr_above_zero:
            inANDdecr_incr_below_zero_and_inANDdecr_above_zero.append(inANDdecr_both_positive_and_negative[i])
        elif my_series2.is_monotonic_decreasing ==True and inANDdecr_both_positive_and_negative[i] not in inANDdecr_inANDdecr_below_zero_and_decr_above_zero:
            inANDdecr_inANDdecr_below_zero_and_decr_above_zero.append(inANDdecr_both_positive_and_negative[i])
        elif my_series2.is_monotonic ==True and inANDdecr_both_positive_and_negative[i] not in inANDdecr_inANDdecr_below_zero_and_incr_above_zero:
            inANDdecr_inANDdecr_below_zero_and_incr_above_zero.append(inANDdecr_both_positive_and_negative[i])
        
    i=i+1

inANDdecr_decr_below_zero_and_decr_above_zero


[]

In [2665]:
inANDdecr_incr_below_zero_and_inANDdecr_above_zero

['EX_co2_e']

In [2666]:
inANDdecr_inANDdecr_below_zero_and_decr_above_zero

['EX_quin_e', 'EX_val__L_e', 'CO2tex']

In [2667]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(inANDdecr_inANDdecr_below_zero_and_decr_above_zero):
    
    i=0
    r_id = inANDdecr_inANDdecr_below_zero_and_decr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 301:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 300
    l= list()
    while i < 600:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1

w5


[]

In [2668]:
one_to_three_data_point_above_zero12

[]

In [2669]:
i=0
one_to_three_data_point_below_zero12= list()
one_to_three_data_point_above_zero12= list()
one_to_three_data_point_below_zero_and_decr_and_stable_above_zero= list()

while i < len(flat_below_zero_and_decr_and_stable_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(flat_below_zero_and_decr_and_stable_above_zero[i]))
    n= 0
    l= list()
    
    while n < 301:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2= 300
    l2= list()
    
    while n2 < 600:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero12:
        
        one_to_three_data_point_below_zero12.append(flat_below_zero_and_decr_and_stable_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero12:
        
        one_to_three_data_point_above_zero12.append(flat_below_zero_and_decr_and_stable_above_zero[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero12

flat_below_zero_and_decr_and_stable_above_zero.remove("Kt3pp")
one_to_three_data_point_below_zero_and_decr_and_stable_above_zero.append("Kt3pp")
one_to_three_data_point_below_zero_and_decr_and_stable_above_zero


['Kt3pp']

In [2670]:
one_to_three_data_point_above_zero12

[]

In [2671]:
incr_and_stable_below_zero_and_inANDdecr_above_zero = [*set(incr_and_stable_below_zero_and_inANDdecr_above_zero)]


In [2672]:
union = pd.Series(np.union1d(incr_and_stable_below_zero_and_mon_decr_above_zero, incr_and_stable_below_zero_and_inANDdecr_above_zero))
intersect = pd.Series(np.intersect1d(incr_and_stable_below_zero_and_mon_decr_above_zero, incr_and_stable_below_zero_and_inANDdecr_above_zero))
bigb2= union[~union.isin(intersect)]


set1= set(incr_and_stable_below_zero_and_inANDdecr_above_zero)
set2= set(intersect)
set3= set1-set2
set3


set()

In [2673]:
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero

['GLUDy', 'PGK', 'CO2tpp', 'PGM']

In [2674]:
len(inANDdecr_and_stable_below_zero_and_decreasing_and_stable_above_zero)+len(increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero)

50

In [2675]:
len(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)

50

In [2676]:
len(one_at_zero)+len(flat_above_zero)+len(flat_below_zero)+len(mon_decr_and_stable_below_zero)+len(mon_incr_and_stable_below_zero)+len(inANDdecr_and_stable_below_zero)+len(mon_decr_and_stable_above_zero)+len(mon_incr_and_stable_above_zero)+len(inANDdecr_and_stable_above_zero)+len(flat_below_zero_and_flat_above_zero)+len(mon_incr_below_zero_and_flat_above_zero)+len(decr_and_stable_below_zero_and_mon_decr_above_zero)+len(incr_and_stable_below_zero_and_mon_decr_above_zero)+len(incr_and_stable_below_zero_and_flat_above_zero)+len(flat_below_zero_and_mon_decr_above_zero)+len(flat_below_zero_and_decr_and_stable_above_zero)+len(decr_and_stable_below_zero_and_flat_above_zero)+len(decr_both_positive_and_negative)+len(incr_both_positive_and_negative)+len(decreasing_and_stable_below_zero_and_decreasing_and_stable_above_zero)+len(increasing_and_stable_below_zero_and_increasing_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero_and_decreasing_above_zero)+len(incr_below_zero_and_inANDdecr_and_stable_above_zero)+len(incr_below_zero_and_incr_and_stable_above_zero)+len(decr_below_zero_and_decr_and_stable_above_zero)+len(incr_below_zero_and_decr_and_stable_above_zero)+len(increasing_below_zero_and_decreasing_above_zero)+len(decr_above_zero)+len(incr_above_zero)+len(one_to_three_data_point_above_zero)+len(inANDdecr_above_zero)+len(incr_below_zero)+len(decr_below_zero)+len(one_to_three_data_point_below_zero)+len(inANDdecr_below_zero)+len(inANDdecr_incr_below_zero_and_decr_above_zero)+len(inANDdecr_incr_below_zero_and_inANDdecr_above_zero)+len(inANDdecr_inANDdecr_below_zero_and_decr_above_zero)+len(one_to_three_data_point_below_zero_and_mon_decr_above_zero)+len(one_to_three_data_point_below_zero_and_decr_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)+len(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)


2726

In [2677]:
len(one_at_zero)+len(flat_above_zero)+len(flat_below_zero)+len(mon_decr_and_stable_below_zero)+len(mon_incr_and_stable_below_zero)+len(inANDdecr_and_stable_below_zero)+len(mon_decr_and_stable_above_zero)+len(mon_incr_and_stable_above_zero)+len(inANDdecr_and_stable_above_zero)+len(flat_below_zero_and_flat_above_zero)+len(mon_incr_below_zero_and_flat_above_zero)+len(decr_and_stable_below_zero_and_mon_decr_above_zero)+len(incr_and_stable_below_zero_and_mon_decr_above_zero)+len(incr_and_stable_below_zero_and_flat_above_zero)+len(flat_below_zero_and_mon_decr_above_zero)+len(flat_below_zero_and_decr_and_stable_above_zero)+len(decr_and_stable_below_zero_and_flat_above_zero)+len(decr_both_positive_and_negative)+len(incr_both_positive_and_negative)+len(decreasing_and_stable_below_zero_and_decreasing_and_stable_above_zero)+len(increasing_and_stable_below_zero_and_increasing_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero_and_decreasing_above_zero)+len(incr_below_zero_and_inANDdecr_and_stable_above_zero)+len(incr_below_zero_and_incr_and_stable_above_zero)+len(decr_below_zero_and_decr_and_stable_above_zero)+len(incr_below_zero_and_decr_and_stable_above_zero)+len(increasing_below_zero_and_decreasing_above_zero)+len(decr_above_zero)+len(incr_above_zero)+len(one_to_three_data_point_above_zero)+len(inANDdecr_above_zero)+len(incr_below_zero)+len(decr_below_zero)+len(one_to_three_data_point_below_zero)+len(inANDdecr_below_zero)+len(inANDdecr_incr_below_zero_and_decr_above_zero)+len(inANDdecr_incr_below_zero_and_inANDdecr_above_zero)+len(inANDdecr_inANDdecr_below_zero_and_decr_above_zero)+len(one_to_three_data_point_below_zero_and_mon_decr_above_zero)+len(one_to_three_data_point_below_zero_and_decr_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)+len(inANDdecr_and_stable_below_zero_and_decreasing_and_stable_above_zero)+len(increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero)


2726

In [2678]:
lt= list(one_at_zero)+list(flat_above_zero)+list(flat_below_zero)+list(mon_decr_and_stable_below_zero)+list(mon_incr_and_stable_below_zero)+list(inANDdecr_and_stable_below_zero)+list(mon_decr_and_stable_above_zero)+list(mon_incr_and_stable_above_zero)+list(inANDdecr_and_stable_above_zero)+list(flat_below_zero_and_flat_above_zero)+list(mon_incr_below_zero_and_flat_above_zero)+list(decr_and_stable_below_zero_and_mon_decr_above_zero)+list(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero)+list(incr_and_stable_below_zero_and_mon_decr_above_zero)+list(incr_and_stable_below_zero_and_flat_above_zero)+list(flat_below_zero_and_mon_decr_above_zero)+list(flat_below_zero_and_decr_and_stable_above_zero)+list(decr_and_stable_below_zero_and_flat_above_zero)+list(decr_both_positive_and_negative)+list(incr_both_positive_and_negative)+list(decreasing_and_stable_below_zero_and_decreasing_and_stable_above_zero)+list(increasing_and_stable_below_zero_and_increasing_and_stable_above_zero)+list(inANDdecr_and_stable_below_zero_and_decreasing_above_zero)+list(incr_below_zero_and_inANDdecr_and_stable_above_zero)+list(incr_below_zero_and_incr_and_stable_above_zero)+list(decr_below_zero_and_decr_and_stable_above_zero)+list(incr_below_zero_and_decr_and_stable_above_zero)+list(increasing_below_zero_and_decreasing_above_zero)+list(decr_above_zero)+list(incr_above_zero)+list(one_to_three_data_point_above_zero)+list(inANDdecr_above_zero)+list(incr_below_zero)+list(decr_below_zero)+list(one_to_three_data_point_below_zero)+list(inANDdecr_below_zero)+list(inANDdecr_incr_below_zero_and_decr_above_zero)+list(inANDdecr_incr_below_zero_and_inANDdecr_above_zero)+list(inANDdecr_inANDdecr_below_zero_and_decr_above_zero)+list(one_to_three_data_point_below_zero_and_mon_decr_above_zero)+list(one_to_three_data_point_below_zero_and_decr_and_stable_above_zero)+list(inANDdecr_and_stable_below_zero_and_decreasing_and_stable_above_zero)+list(increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero)


In [2679]:
l1=[]
for i in lt:
    if i not in l1:
        l1.append(i)
    else:
        print(i,end=' ')


In [2680]:
csvFile = pd.read_csv('all reactions.csv')
csvFile["Reaction names"]

rl= list(csvFile["Reaction names"])

set1= set(rl)
set2= set(lt)
set3= set1-set2
set3


set()

In [2681]:
incr_below_zero_and_decr_and_stable_above_zero= list(incr_below_zero_and_decr_and_stable_above_zero)+list(inANDdecr_incr_below_zero_and_decr_above_zero)


In [2598]:
D= pd.DataFrame (one_at_zero, columns = ['Reaction names'])
D1= pd.DataFrame (flat_above_zero, columns = ['Reaction names'])
D2= pd.DataFrame (flat_below_zero, columns = ['Reaction names'])
D3= pd.DataFrame (mon_incr_and_stable_above_zero, columns = ['Reaction names'])
D4= pd.DataFrame (mon_decr_and_stable_above_zero, columns = ['Reaction names'])
D5= pd.DataFrame (inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D6= pd.DataFrame (inANDdecr_and_stable_below_zero, columns = ['Reaction names'])
D7= pd.DataFrame (mon_incr_and_stable_below_zero, columns = ['Reaction names'])
D8= pd.DataFrame (mon_decr_and_stable_below_zero, columns = ['Reaction names'])
D9= pd.DataFrame (flat_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D10= pd.DataFrame (mon_incr_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D11= pd.DataFrame (decr_and_stable_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D12= pd.DataFrame (incr_and_stable_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D13= pd.DataFrame (incr_and_stable_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D14= pd.DataFrame (flat_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D15= pd.DataFrame (flat_below_zero_and_decr_and_stable_above_zero, columns = ['Reaction names'])
D16= pd.DataFrame (decr_and_stable_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D17= pd.DataFrame (decr_both_positive_and_negative, columns = ['Reaction names'])
D18= pd.DataFrame (incr_both_positive_and_negative, columns = ['Reaction names'])
D19= pd.DataFrame (decreasing_and_stable_below_zero_and_decreasing_and_stable_above_zero, columns = ['Reaction names'])
D20= pd.DataFrame (increasing_and_stable_below_zero_and_increasing_and_stable_above_zero, columns = ['Reaction names'])
D21= pd.DataFrame (inANDdecr_and_stable_below_zero_and_decreasing_and_stable_above_zero, columns = ['Reaction names'])
D22= pd.DataFrame (inANDdecr_and_stable_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D23= pd.DataFrame (incr_below_zero_and_inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D24= pd.DataFrame (incr_below_zero_and_incr_and_stable_above_zero, columns = ['Reaction names'])
D25= pd.DataFrame (decr_below_zero_and_decr_and_stable_above_zero, columns = ['Reaction names'])
D26= pd.DataFrame (incr_below_zero_and_decr_and_stable_above_zero, columns = ['Reaction names'])
D27= pd.DataFrame (increasing_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D28= pd.DataFrame (decr_above_zero, columns = ['Reaction names'])
D29= pd.DataFrame (incr_above_zero, columns = ['Reaction names'])
D30= pd.DataFrame (one_to_three_data_point_above_zero, columns = ['Reaction names'])
D31= pd.DataFrame (inANDdecr_above_zero, columns = ['Reaction names'])
D32= pd.DataFrame (incr_below_zero, columns = ['Reaction names'])
D33= pd.DataFrame (decr_below_zero, columns = ['Reaction names'])
D34= pd.DataFrame (one_to_three_data_point_below_zero, columns = ['Reaction names'])
D35= pd.DataFrame (inANDdecr_below_zero, columns = ['Reaction names'])
D37= pd.DataFrame (inANDdecr_incr_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])
D38= pd.DataFrame (inANDdecr_inANDdecr_below_zero_and_decr_above_zero, columns = ['Reaction names'])
D39= pd.DataFrame (one_to_three_data_point_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D40= pd.DataFrame (one_to_three_data_point_below_zero_and_decr_and_stable_above_zero, columns = ['Reaction names'])
D41= pd.DataFrame (inANDdecr_and_stable_below_zero_and_mon_decr_above_zero, columns= ['Reaction names'])
D42= pd.DataFrame (increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero, columns= ['Reaction names'])


In [2599]:
D.to_csv("NEW one feasible value at zero reactions -30 to 30.csv")
D1.to_csv("NEW flat_above_zero -30 to 30.csv")
D2.to_csv("NEW flat_below_zero -30 to 30.csv")
D3.to_csv("NEW increasing and stable above zero -30 to 30.csv")
D4.to_csv("NEW decreasing and stable above zero -30 to 30.csv")
D5.to_csv("NEW increasing and decreasing and stable above zero -30 to 30.csv")
D6.to_csv("NEW increasing and decreasing and stable below zero -30 to 30.csv")
D7.to_csv("NEW decreasing and stable below zero -30 to 30.csv")
D8.to_csv("NEW increasing and stable below zero -30 to 30.csv")
D9.to_csv("NEW flat below and flat above zero -30 to 30.csv")
D10.to_csv("NEW decreasing below zero and flat above zero -30 to 30.csv")
D11.to_csv("NEW increasing and stable below zero and decreasing above zero -30 to 30.csv")
D12.to_csv("NEW decreasing and stable below zero and decreasing above zero -30 to 30.csv")
D13.to_csv("NEW decreasing and stable below and flat above zero -30 to 30.csv")
D14.to_csv("NEW flat below zero and decreasing above zero -30 to 30.csv")
D15.to_csv("NEW flat below zero and decreasing and stable above zero -30 to 30.csv")
D16.to_csv("NEW increasing and stable below zero and flat above zero -30 to 30.csv")
D17.to_csv("NEW increasing below zero and decreasing above zero -30 to 30.csv")
D18.to_csv("NEW decreasing below zero and increasing above zero -30 to 30.csv")
D19.to_csv("NEW increasing and stable below zero and decreasing and stable above zero -30 to 30.csv")
D20.to_csv("NEW decreasing and stable below zero and increasing and stable above zero -30 to 30.csv")
D21.to_csv("NEW inANDdecr_and_stable_below_zero_and_decreasing_and_stable_above_zero -30 to 30.csv")
D22.to_csv("NEW inANDdecr_and_stable_below_zero_and_decreasing_above_zero -30 to 30.csv")
D23.to_csv("NEW decreasing below zero and inANDdecr and stable above zero -30 to 30.csv")
D24.to_csv("NEW decreasing below zero and increasing and stable above zero -30 to 30.csv")
D25.to_csv("NEW increasing below zero and decreasing and stable above zero -30 to 30.csv")
D26.to_csv("NEW decreasing below zero and decreasing and stable above zero -30 to 30.csv")
D27.to_csv("NEW decreasing below zero and decreasing above zero -30 to 30.csv")
D28.to_csv("NEW decreasing above zero -30 to 30.csv")
D29.to_csv("NEW increasing above zero -30 to 30.csv")
D30.to_csv("NEW one to three data point above zero -30 to 30.csv")
D31.to_csv("NEW inANDdecr_above_zero -30 to 30.csv")
D32.to_csv("NEW decreasing below zero -30 to 30.csv")
D33.to_csv("NEW increasing below zero -30 to 30.csv")
D34.to_csv("NEW one to three data point below zero -30 to 30.csv")
D35.to_csv("NEW inANDdecr_below_zero -30 to 30.csv")
D37.to_csv("NEW decreasing below zero and increasing and decreasing above zero -30 to 30.csv")
D38.to_csv("NEW increasing and decreasing below zero and decreasing above zero -30 to 30.csv")
D39.to_csv("NEW one_to_three_data_point_below_zero_and_mon_decr_above_zero -30 to 30.csv")
D40.to_csv("NEW one_to_three_data_point_below_zero_and_decr_and_stable_above_zero -30 to 30.csv")
D41.to_csv("NEW increasing and decreasing and stable below zero and decreasing above zero -30 to 30.csv")
D42.to_csv("NEW decreasing and stable below zero and decreasing and stable above zero -30 to 30.csv")


In [1097]:
Dtotal= pd.DataFrame (one_at_zero, columns = ['Reaction names'])
Dtotal= Dtotal.append(pd.DataFrame(flat_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(flat_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(mon_decr_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(mon_incr_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(mon_decr_and_stable_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(mon_incr_and_stable_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(F_below_zero_and_F_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_and_stable_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(decreasing_and_stable_below_zero_and_decreasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(flat_below_zero_and_decreasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(decreasing_and_stable_below_zero_and_decreasing_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(flat_below_zero_and_decreasing_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_and_stable_below_zero_and_decreasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_below_zero_and_flat_above_zero))   
Dtotal= Dtotal.append(pd.DataFrame(increasing_and_stable_below_zero_and_increasing_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_and_stable_below_zero_and_flat_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_and_stable_below_zero_and_decreasing_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_below_zero_and_decreasing_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_and_stable_below_zero_and_inANDdecr_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(decr_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(incr_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(one_to_three_data_point_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(decr_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(incr_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(one_to_three_data_point_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(decr_both_positive_and_negative))
Dtotal= Dtotal.append(pd.DataFrame(incr_both_positive_and_negative))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_incr_below_zero_and_inANDdecr_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_incr_below_zero_and_decr_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_inANDdecr_below_zero_and_decr_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_and_stable_below_zero_and_decreasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(one_to_three_data_point_below_zero_and_decreasing_above_zero))
              
Dtotal.to_csv("NEW all reactions -30 to 30 lactose.csv")


In [2160]:
one_at_zero
flat_above_zero
flat_below_zero
mon_decr_and_stable_below_zero
mon_incr_and_stable_below_zero
inANDdecr_and_stable_below_zero
mon_decr_and_stable_above_zero
mon_incr_and_stable_above_zero
inANDdecr_and_stable_above_zero


flat_below_zero_and_flat_above_zero
mon_incr_below_zero_and_flat_above_zero
decr_and_stable_below_zero_and_mon_decr_above_zero
incr_and_stable_below_zero_and_mon_decr_above_zero

incr_and_stable_below_zero_and_flat_above_zero
flat_below_zero_and_mon_decr_above_zero
flat_below_zero_and_decr_and_stable_above_zero
decr_and_stable_below_zero_and_flat_above_zero
decr_both_positive_and_negative
incr_both_positive_and_negative
decreasing_and_stable_below_zero_and_decreasing_and_stable_above_zero
increasing_and_stable_below_zero_and_increasing_and_stable_above_zero
inANDdecr_and_stable_below_zero_and_decreasing_and_stable_above_zero
increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero
inANDdecr_and_stable_below_zero_and_decreasing_above_zero
incr_below_zero_and_inANDdecr_and_stable_above_zero
incr_below_zero_and_incr_and_stable_above_zero
decr_below_zero_and_decr_and_stable_above_zero
incr_below_zero_and_decr_and_stable_above_zero
increasing_below_zero_and_decreasing_above_zero
decr_above_zero
incr_above_zero
one_to_three_data_point_above_zero
inANDdecr_above_zero
incr_below_zero
decr_below_zero
one_to_three_data_point_below_zero
inANDdecr_below_zero
inANDdecr_incr_below_zero_and_decr_above_zero
inANDdecr_incr_below_zero_and_inANDdecr_above_zero
inANDdecr_inANDdecr_below_zero_and_decr_above_zero

one_to_three_data_point_below_zero_and_mon_decr_above_zero
one_to_three_data_point_below_zero_and_decr_and_stable_above_zero
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero


['GLUDy', 'PGK', 'CO2tpp', 'PGM']

In [ ]:
flat_above_zero= list(flat_above_zero) + list(flat3_above_zero)
flat_above_zero